## Setting up OpenAI Model

In [1]:
# Install llama-index libraries
!pip install llama-index
%pip install llama-index-vector-stores-pinecone
!pip install wikipedia
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-openai
!pip install llama-index-embeddings-huggingface
# Install openai library
!pip install openai

In [2]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "ADD_YOUR_KEY_HERE"

client = openai.OpenAI()

In [3]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Setting up RAG Model

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [5]:
# Importing HuggingFace Embedding
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.chunk_size = 150  # set chunk size
Settings.chunk_overlap = 25  # overlap

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# Use wikipedia library to load data related to aerospace materials
from llama_index.readers.wikipedia import WikipediaReader

materials = ["2024 Aluminum Alloy Plate"]
wiki_docs = WikipediaReader().load_data(pages=materials)

In [7]:
# store docs into vector DB
index = VectorStoreIndex.from_documents(wiki_docs)

In [8]:
# top number of chunks to retrieve
top_k = 3

# vector retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [9]:
# query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

## Testing RAG Model

In [10]:
# query documents
query = "What is the lowest ftu allowable for 2024 aluminum alloy plate?"
response = query_engine.query(query)

In [11]:
# reformat response and printing top chunks
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
the Pioneer plaque
the secondary chambers and baffle systems in firearm sound suppressors (primarily pistol suppressors for reduced weight and improved mechanical functionality), while the primary expansion chambers usually require 17-4PH or 303 stainless steel or titanium.
the upper and lower receivers of many non mil-spec AR-15 rifle variants.
many aluminium docks and gangways, welded into place.
material used in some ultra-high vacuum (UHV) chambers
many parts for remote controlled model aircraft, notably helicopter rotor components.
large amateur radio antennas.
fire department rescue ladders


=== Welding ===
6061 is highly weldable, for example using tungsten inert gas welding (TIG) or metal inert gas welding (MIG).

T651 temper has similar mechanical properties.
The typical value for thermal conductivity for 6061-T6 at 25 °C (77 °F) is around 152 W/m K.
The fatigue limit under cyclic load is 97 MPa (14 ksi) for 500,000,000 completely reversed cycles using a standard RR 

In [13]:
# Prompt response without using RAG
message=[
        {
            "role": "system",
            "content": "You are a chatbot knowledgeable in Aluminum Design Allowables",
        },
        {
            "role": "user",
            "content": "What is the lowest ftu allowable for 2024 aluminum alloy plate at stock thickness between 0.250 and 0.499?"
        }]

response = client.chat.completions.create(model="gpt-4o-mini",
                                          messages = message)
response.choices[0].message.content

'For 2024 aluminum alloy, the allowable fatigue ultimate tensile strength (ftu) can vary based on specific conditions and heat treatment, such as T3 or T4 temper. However, for 2024-T3, which is a common temper used for this alloy, the typical minimum allowable values for the ultimate tensile strength are around 70 ksi (approximately 483 MPa). \n\nIn specific design codes or applications, the allowable values may be determined through further detailed analysis or tests. Always refer to the pertinent material data sheets and engineering handbooks such as the Aluminum Association or the aerospace industry specifications for the most accurate and context-specific values.'

In [ ]:
# Using RAG
rag_prompt = lambda context, comment: f"""
{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
"""
comment = "What accident happened on 12/20/2011 in Morristown, NJ?"
prompt = rag_prompt(context, comment)

message=[
        {
            "role": "system",
            "content": "You are a chatbot knowledgeable in FAA documents",
        },
        {
            "role": "user",
            "content": prompt
        }]

response = client.chat.completions.create(model=deployment_name,
                                          messages = message)
response.choices[0].message.content